### Load libs

In [1]:
import pandas as pd
import json
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize()


INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
n = 500
pd.set_option('display.max_columns', n)
pd.set_option('display.max_rows', n)
pd.set_option('display.max_colwidth', -1)

/anaconda3/envs/kn/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


### Load data

In [3]:
term = 'AmeriCredit Automobile Receivables Trust 2017-1 Data Tape'


In [4]:
# load abs
folder = 'data/combined/'
file = '{}.csv'.format(term)
path = folder + file
data = pd.read_csv(path)
data.shape


/anaconda3/envs/kn/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (38,39,57) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1511394, 73)

In [5]:
# load mapper
m_folder = 'data/dictionary/mapper/'
m_file = 'mapper.json'
m_path = m_folder + m_file
with open(m_path) as f:
    mapper = json.load(f)
m_cols = list(mapper.keys())
    

### Formatting

In [6]:
def convert_id(row, column):
    
    """
    Convert ids
    """
    
    init = str(row[column])
    
    cleaned = init.replace('=', '').replace('"', '').strip()
    
    return cleaned
    

In [7]:
def reorder_date(row, column):
    
    """
    Reorder date
    """
    
    init = str(row[column])
    if init != '-':
        if '/' not in init:
            y = init[6:10]
            m = init[0:2]
            d = init[3:5]
            date = y + '-' + m + '-' + d
        elif '/' in init:
            y = init[3:7]
            m = init[0:2]
            date = y + '-' + m
    else:
        date = ''
    
    return date
        

In [8]:
data['ID'] = data.parallel_apply(convert_id, args = ('assetNumber', ), axis = 1)


In [9]:
# date cols
date_cols = ['reportingPeriodBeginningDate', 'reportingPeriodEndingDate', 'originationDate', 'loanMaturityDate', 'originalFirstPaymentDate', 'interestPaidThroughDate', 'zeroBalanceEffectiveDate', 'mostRecentServicingTransferReceivedDate', 'DemandResolutionDate'] 


In [10]:
for col in date_cols:
    print(col)
    data['{}R'.format(col)] = data.parallel_apply(reorder_date, args = (col, ), axis = 1)
    

reportingPeriodBeginningDate
reportingPeriodEndingDate
originationDate
loanMaturityDate
originalFirstPaymentDate
interestPaidThroughDate
zeroBalanceEffectiveDate
mostRecentServicingTransferReceivedDate
DemandResolutionDate


In [11]:
replacer_cols = ['originalLoanAmount', 'originalLoanTerm', 'originalInterestRatePercentage', 'gracePeriodNumber', 'obligorCreditScore', 'paymentToIncomePercentage', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'scheduledInterestAmount', 'scheduledPrincipalAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'repurchaseAmount', 'chargedoffPrincipalAmount', 'recoveredAmount', 'repossessedProceedsAmount']    


In [12]:
data[replacer_cols] = data[replacer_cols].replace('-', np.nan)

In [13]:
# clean cols
clean_cols = ['currentDelinquencyStatus']
for col in clean_cols:
    data[col] = data[col].str.strip()
    data[col] = data[col].astype(float)
    

In [14]:
def replace_val(row, column):
    
    """
    Replace numeric values
    """
    
    init = str(row[column]).strip().replace(';', '')
    if init in ['0', '1', '2', '3', '4', '5', '98', '99']:
        mapped = mapper[column][init]
        return mapped
    else:
        if init[0] in ['0', '1', '2', '3', '4', '5']:
            use = init[0]
        elif init == '-':
            use_keys = list(mapper[column].keys())
            if '98' in use_keys:
                use = '98'
            elif '99' in use_keys:
                use = '99'
        else:
            use = init
        mapped = mapper[column][use]
    return mapped
    

In [15]:
for col in m_cols:
    print(col)
    new_col = col + 'M'
    data[new_col] = data.parallel_apply(replace_val, args = (col, ), axis = 1)
    

interestCalculationTypeCode
originalInterestRateTypeCode
paymentTypeCode
subvented
vehicleNewUsedCode
vehicleTypeCode
vehicleValueSourceCode
obligorIncomeVerificationLevelCode
obligorEmploymentVerificationCode
servicingAdvanceMethodCode
zeroBalanceCode
assetSubjectDemandStatusCode
repurchaseReplacementReasonCode
modificationTypeCode


In [16]:
def acct_status(row, b_col, e_col, zero_col, thresh):
    
    """
    Create karus account status
    """
    
    b = float(row[b_col])
    e = float(row[e_col])
    z = str(row[zero_col])
    
    if z in ['Charged-off', 'Repurchased or Replaced']:
        res = z
        return res
    if b < thresh and e < thresh:
        res = 'Prepaid or Matured'
        return res
    if z in ['Unavailable', 'Prepaid or Matured']:
        res = z
        return res
    

In [17]:
b_col = 'reportingPeriodBeginningLoanBalanceAmount'
e_col = 'nextReportingPeriodPaymentAmountDue'
z_col = 'zeroBalanceCodeM'
thresh = 50


In [18]:
data['accountStatus'] = data.parallel_apply(acct_status, args = (b_col, e_col, z_col, thresh, ), axis = 1)

In [19]:
data['accountStatus'].value_counts()

Unavailable                1458409
Charged-off                26714  
Prepaid or Matured         26234  
Repurchased or Replaced    37     
Name: accountStatus, dtype: int64

### Application

In [20]:
all_ids = list(data['ID'].unique())
print_vals = list(range(0, len(all_ids), 100))


In [21]:
loc_cols = list(set(['originatorName', 'primaryLoanServicerName', 'originationDateR', 'originalLoanAmount', 'originalLoanTerm', 'loanMaturityDateR', 'originalInterestRatePercentage', 'interestCalculationTypeCodeM', 'originalInterestRateTypeCodeM', 'originalInterestOnlyTermNumber', 'originalFirstPaymentDateR', 'underwritingIndicator', 'paymentTypeCodeM', 'vehicleManufacturerName', 'vehicleModelName', 'vehicleNewUsedCodeM', 'vehicleModelYear', 'vehicleTypeCodeM', 'vehicleValueAmount', 'vehicleValueSourceCodeM', 'obligorCreditScoreType', 'obligorIncomeVerificationLevelCodeM', 'obligorEmploymentVerificationCodeM', 'coObligorIndicator', 'paymentToIncomePercentage', 'obligorGeographicLocation', 'assetAddedIndicator', 'reportingPeriodModificationIndicator', 'servicingAdvanceMethodCodeM', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'scheduledInterestAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'assetSubjectDemandIndicator', 'zeroBalanceEffectiveDateR']))         
min_max_cols = list(set(['reportingPeriodBeginningDateR', 'reportingPeriodEndingDateR', 'remainingTermToMaturityNumber', 'obligorCreditScore', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue']))
max_cols = list(set(['gracePeriodNumber', 'interestPaidThroughDateR', 'mostRecentServicingTransferReceivedDateR', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'currentDelinquencyStatus']))
all_vals = list(set(['subventedM', 'assetSubjectDemandIndicator', 'assetSubjectDemandStatusCodeM', 'repurchaserName', 'repurchaseReplacementReasonCodeM', 'modificationTypeCodeM', 'repossessedIndicator', 'zeroBalanceCodeM', 'accountStatus']))
sum_cols = list(set(['servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'repurchaseAmount', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'repossessedProceedsAmount', 'gracePeriodNumber']))
event_cols = list(set(['accountStatus', 'zeroBalanceCodeM', 'chargedoffPrincipalAmount', 'reportingPeriodBeginningDateR', 'reportingPeriodEndingDateR']))


In [22]:
id_col = 'ID'
#_id = '0001694010 - 000010'
status_col = 'accountStatus'
values = ['Charged-off', 'Prepaid or Matured', 'Repurchased or Replaced']


In [52]:
holder = []
counter = 0

for _id in all_ids:
    #print(_id)
    counter = counter + 1
    if counter in print_vals:
        print(counter, counter/len(data))
        print('------------------------------')
    
    df = data[data[id_col] == _id].reset_index(drop = True)
    df = df.sort_values('reportingPeriodBeginningDateR', ascending = False)

    # dict
    account_dict = {}
    account_dict['id'] = _id
    account_dict['records'] = len(df)

    # grab current info
    for col in loc_cols:
        account_dict['{}LocCurrent'.format(col)] = df[col].iloc[0]
    for col in min_max_cols:
        _min = df[col].min()
        _max = df[col].max()
        account_dict['{}MinCurrent'.format(col)] = _min
        account_dict['{}MaxCurrent'.format(col)] = _max
    for col in max_cols:
        account_dict['{}MaxCurrent'.format(col)] = df[col].max()
    for col in all_vals:
        vals = list(df[col].unique())
        use_vals = ' | '.join(str(val) for val in vals)
        account_dict['{}ValsCurrent'.format(col)] = use_vals
    for col in sum_cols:
        account_dict['{}SumCurrent'.format(col)] = df[col].sum()

    # event
    init_vals = list(df[status_col].unique())
    inter = list(set(values).intersection(init_vals))
    if len(inter) > 0:
        account_dict['eventOccured'] = 1
        n = df[status_col].where(df[status_col].isin(values)).last_valid_index()
        single = df.loc[[n]]
        for col in event_cols:
            account_dict['{}Event'.format(col)] = single[col].iloc[0]

        # prior to event
        init = n+1
        sub = df[init:len(df)]
        account_dict['priorHistory'] = len(sub)
        sub.reset_index(drop = True, inplace = True)
        if len(sub) > 0:
            for col in loc_cols:
                account_dict['{}LocPrior'.format(col)] = sub[col].iloc[0]
            for col in min_max_cols:
                _min = sub[col].min()
                _max = sub[col].max()
                account_dict['{}MinPrior'.format(col)] = _min
                account_dict['{}MaxPrior'.format(col)] = _max
            for col in max_cols:
                account_dict['{}MaxPrior'.format(col)] = sub[col].max()
            for col in all_vals:
                vals = list(sub[col].unique())
                use_vals = ' | '.join(str(val) for val in vals)
                account_dict['{}ValsPrior'.format(col)] = use_vals
            for col in sum_cols:
                account_dict['{}SumPrior'.format(col)] = sub[col].sum()
        else:
            for col in loc_cols:
                account_dict['{}LocPrior'.format(col)] = df[col].iloc[0]
            for col in min_max_cols:
                _min = df[col].min()
                _max = df[col].max()
                account_dict['{}MinPrior'.format(col)] = _min
                account_dict['{}MaxPrior'.format(col)] = _max
            for col in max_cols:
                account_dict['{}MaxPrior'.format(col)] = df[col].max()
            for col in all_vals:
                vals = list(df[col].unique())
                use_vals = ' | '.join(str(val) for val in vals)
                account_dict['{}ValsPrior'.format(col)] = use_vals
            for col in sum_cols:
                account_dict['{}SumPrior'.format(col)] = df[col].sum()
    else:
        account_dict['eventOccured'] = 0
        account_dict['priorHistory'] = len(df)
        for col in event_cols:
            account_dict['{}Event'.format(col)] = np.nan
        for col in loc_cols:
            account_dict['{}LocPrior'.format(col)] = df[col].iloc[0]
        for col in min_max_cols:
            _min = df[col].min()
            _max = df[col].max()
            account_dict['{}MinPrior'.format(col)] = _min
            account_dict['{}MaxPrior'.format(col)] = _max
        for col in max_cols:
            account_dict['{}MaxPrior'.format(col)] = df[col].max()
        for col in all_vals:
            vals = list(df[col].unique())
            use_vals = ' | '.join(str(val) for val in vals)
            account_dict['{}ValsPrior'.format(col)] = use_vals
        for col in sum_cols:
            account_dict['{}SumPrior'.format(col)] = df[col].sum()
            
    holder.append(account_dict)
        

100 6.616408428245712e-05
------------------------------
200 0.00013232816856491425
------------------------------
300 0.00019849225284737136
------------------------------


KeyboardInterrupt: 

In [53]:
master_df = pd.DataFrame(holder)

### Debugging

In [171]:
_id = '0001694010 - 006770'

In [172]:
df = data[data[id_col] == _id].reset_index(drop = True)
df = df.sort_values('reportingPeriodBeginningDateR', ascending = False)

# dict
account_dict = {}
account_dict['id'] = _id
account_dict['records'] = len(df)

# grab current info
for col in loc_cols:
    account_dict['{}LocCurrent'.format(col)] = df[col].iloc[0]
for col in min_max_cols:
    _min = df[col].min()
    _max = df[col].max()
    account_dict['{}MinCurrent'.format(col)] = _min
    account_dict['{}MaxCurrent'.format(col)] = _max
for col in max_cols:
    account_dict['{}MaxCurrent'.format(col)] = df[col].max()
for col in all_vals:
    vals = list(df[col].unique())
    use_vals = ' | '.join(str(val) for val in vals)
    account_dict['{}ValsCurrent'.format(col)] = use_vals
for col in sum_cols:
    account_dict['{}SumCurrent'.format(col)] = df[col].sum()

# event
init_vals = list(df[status_col].unique())
inter = list(set(values).intersection(init_vals))
if len(inter) > 0:
    account_dict['eventOccured'] = 1
    n = df[status_col].where(df[status_col].isin(values)).last_valid_index()
    n_bool = True
    single = df.loc[[n]]
    for col in event_cols:
        account_dict['{}Event'.format(col)] = single[col].iloc[0]

    # prior to event
    init = n+1
    sub = df[init:len(df)]
    sub.reset_index(drop = True, inplace = True)
    account_dict['priorHistory'] = len(sub)
    sub_bool = True
    if len(sub) > 0:
        for col in loc_cols:
            account_dict['{}LocPrior'.format(col)] = sub[col].iloc[0]
        for col in min_max_cols:
            _min = sub[col].min()
            _max = sub[col].max()
            account_dict['{}MinPrior'.format(col)] = _min
            account_dict['{}MaxPrior'.format(col)] = _max
        for col in max_cols:
            account_dict['{}MaxPrior'.format(col)] = sub[col].max()
        for col in all_vals:
            vals = list(sub[col].unique())
            use_vals = ' | '.join(str(val) for val in vals)
            account_dict['{}ValsPrior'.format(col)] = use_vals
        for col in sum_cols:
            account_dict['{}SumPrior'.format(col)] = sub[col].sum()
    else:
        for col in loc_cols:
            account_dict['{}LocPrior'.format(col)] = df[col].iloc[0]
        for col in min_max_cols:
            _min = df[col].min()
            _max = df[col].max()
            account_dict['{}MinPrior'.format(col)] = _min
            account_dict['{}MaxPrior'.format(col)] = _max
        for col in max_cols:
            account_dict['{}MaxPrior'.format(col)] = df[col].max()
        for col in all_vals:
            vals = list(df[col].unique())
            use_vals = ' | '.join(str(val) for val in vals)
            account_dict['{}ValsPrior'.format(col)] = use_vals
        for col in sum_cols:
            account_dict['{}SumPrior'.format(col)] = df[col].sum()
else:
    account_dict['eventOccured'] = 0
    account_dict['priorHistory'] = len(df)
    sub_bool = False
    n_bool = False
    for col in event_cols:
        account_dict['{}Event'.format(col)] = np.nan
    for col in loc_cols:
        account_dict['{}LocPrior'.format(col)] = df[col].iloc[0]
    for col in min_max_cols:
        _min = df[col].min()
        _max = df[col].max()
        account_dict['{}MinPrior'.format(col)] = _min
        account_dict['{}MaxPrior'.format(col)] = _max
    for col in max_cols:
        account_dict['{}MaxPrior'.format(col)] = df[col].max()
    for col in all_vals:
        vals = list(df[col].unique())
        use_vals = ' | '.join(str(val) for val in vals)
        account_dict['{}ValsPrior'.format(col)] = use_vals
    for col in sum_cols:
        account_dict['{}SumPrior'.format(col)] = df[col].sum()
        
        

In [173]:
sub_bool

True

In [174]:
n_bool

True

### Validate results

In [175]:
loc_cols = list(set(['originatorName', 'primaryLoanServicerName', 'originationDateR', 'originalLoanAmount', 'originalLoanTerm', 'loanMaturityDateR', 'originalInterestRatePercentage', 'interestCalculationTypeCodeM', 'originalInterestRateTypeCodeM', 'originalInterestOnlyTermNumber', 'originalFirstPaymentDateR', 'underwritingIndicator', 'paymentTypeCodeM', 'vehicleManufacturerName', 'vehicleModelName', 'vehicleNewUsedCodeM', 'vehicleModelYear', 'vehicleTypeCodeM', 'vehicleValueAmount', 'vehicleValueSourceCodeM', 'obligorCreditScoreType', 'obligorIncomeVerificationLevelCodeM', 'obligorEmploymentVerificationCodeM', 'coObligorIndicator', 'paymentToIncomePercentage', 'obligorGeographicLocation', 'assetAddedIndicator', 'reportingPeriodModificationIndicator', 'servicingAdvanceMethodCodeM', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'scheduledInterestAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'assetSubjectDemandIndicator', 'zeroBalanceEffectiveDateR']))         
min_max_cols = list(set(['reportingPeriodBeginningDateR', 'reportingPeriodEndingDateR', 'remainingTermToMaturityNumber', 'obligorCreditScore', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue']))
max_cols = list(set(['gracePeriodNumber', 'interestPaidThroughDateR', 'mostRecentServicingTransferReceivedDateR', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'currentDelinquencyStatus']))
all_vals = list(set(['subventedM', 'assetSubjectDemandIndicator', 'assetSubjectDemandStatusCodeM', 'repurchaserName', 'repurchaseReplacementReasonCodeM', 'modificationTypeCodeM', 'repossessedIndicator', 'zeroBalanceCodeM', 'accountStatus']))
sum_cols = list(set(['servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'repurchaseAmount', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'repossessedProceedsAmount', 'gracePeriodNumber']))
event_cols = list(set(['accountStatus', 'zeroBalanceCodeM', 'chargedoffPrincipalAmount', 'reportingPeriodBeginningDateR', 'reportingPeriodEndingDateR']))


In [176]:
res_dict = {}

In [177]:
for col in loc_cols:
    
    cor = col + 'LocCurrent'
    val = account_dict[cor]
    comp = df[col].iloc[0]
    
    if str(val) == str(comp):
        res = 'Match'
    else:
        res = 'No match'
    
    if res == 'No match':
        print('NO MATCH!!!!!!!')
    
    res_dict[cor] = {'res': res, 'val': val}
    

In [178]:
for col in loc_cols:
    
    if sub_bool == True:
        use_df = sub
    elif sub_bool == False:
        use_df = df
    
    cor = col + 'LocPrior'
    val = account_dict[cor]
    comp = use_df[col].iloc[0]
    
    if str(val) == str(comp):
        res = 'Match'
    else:
        res = 'No match'
        
    if res == 'No match':
        print('NO MATCH!!!!!!!')
    
    res_dict[cor] = {'res': res, 'val': val}
    

In [179]:
for col in event_cols:
    
    cor = col + 'Event'
    val = account_dict[cor]
    
    if n_bool == True:
        comp = single[col].iloc[0]
    else:
        comp = str(np.nan)
    
    if str(val) == str(comp):
        res = 'Match'
    else:
        res = 'No match'
    
    if res == 'No match':
        print('NO MATCH!!!!!!!')
    
    res_dict[cor] = {'res': res, 'val': val}

In [180]:
res_dict

{'assetAddedIndicatorLocCurrent': {'res': 'Match', 'val': False},
 'loanMaturityDateRLocCurrent': {'res': 'Match', 'val': '2017-12'},
 'vehicleValueAmountLocCurrent': {'res': 'Match', 'val': 27700.0},
 'nextReportingPeriodPaymentAmountDueLocCurrent': {'res': 'Match', 'val': 0.0},
 'originalLoanAmountLocCurrent': {'res': 'Match', 'val': 30246.52},
 'reportingPeriodScheduledPaymentAmountLocCurrent': {'res': 'Match',
  'val': 773.74},
 'primaryLoanServicerNameLocCurrent': {'res': 'Match', 'val': 'GM FINANCIAL'},
 'reportingPeriodActualEndBalanceAmountLocCurrent': {'res': 'Match',
  'val': 0.0},
 'reportingPeriodModificationIndicatorLocCurrent': {'res': 'Match',
  'val': False},
 'originalInterestOnlyTermNumberLocCurrent': {'res': 'Match', 'val': 0},
 'paymentTypeCodeMLocCurrent': {'res': 'Match', 'val': 'Monthly'},
 'originalLoanTermLocCurrent': {'res': 'Match', 'val': 61},
 'obligorGeographicLocationLocCurrent': {'res': 'Match', 'val': 'LA'},
 'originalInterestRateTypeCodeMLocCurrent': {

In [181]:
df

,assetTypeNumber,assetNumber,reportingPeriodBeginningDate,reportingPeriodEndingDate,originatorName,originationDate,originalLoanAmount,originalLoanTerm,loanMaturityDate,originalInterestRatePercentage,interestCalculationTypeCode,originalInterestRateTypeCode,originalInterestOnlyTermNumber,originalFirstPaymentDate,underwritingIndicator,gracePeriodNumber,paymentTypeCode,subvented,vehicleManufacturerName,vehicleModelName,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,vehicleValueSourceCode,obligorCreditScoreType,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,obligorGeographicLocation,assetAddedIndicator,remainingTermToMaturityNumber,reportingPeriodModificationIndicator,servicingAdvanceMethodCode,reportingPeriodBeginningLoanBalanceAmount,nextReportingPeriodPaymentAmountDue,reportingPeriodInterestRatePercentage,nextInterestRatePercentage,servicingFeePercentage,servicingFlatFeeAmount,otherServicerFeeRetainedByServicer,otherAssessedUncollectedServicerFeeAmount,scheduledInterestAmount,scheduledPrincipalAmount,otherPrincipalAdjustmentAmount,reportingPeriodActualEndBalanceAmount,reportingPeriodScheduledPaymentAmount,totalActualAmountPaid,actualInterestCollectedAmount,actualPrincipalCollectedAmount,actualOtherCollectedAmount,servicerAdvancedAmount,interestPaidThroughDate,zeroBalanceEffectiveDate,zeroBalanceCode,currentDelinquencyStatus,primaryLoanServicerName,mostRecentServicingTransferReceivedDate,assetSubjectDemandIndicator,assetSubjectDemandStatusCode,repurchaseAmount,DemandResolutionDate,repurchaserName,repurchaseReplacementReasonCode,chargedoffPrincipalAmount,recoveredAmount,modificationTypeCode,paymentExtendedNumber,repossessedIndicator,repossessedProceedsAmount,dataset_name,ID,reportingPeriodBeginningDateR,reportingPeriodEndingDateR,originationDateR,loanMaturityDateR,originalFirstPaymentDateR,interestPaidThroughDateR,zeroBalanceEffectiveDateR,mostRecentServicingTransferReceivedDateR,DemandResolutionDateR,interestCalculationTypeCodeM,originalInterestRateTypeCodeM,paymentTypeCodeM,subventedM,vehicleNewUsedCodeM,vehicleTypeCodeM,vehicleValueSourceCodeM,obligorIncomeVerificationLevelCodeM,obligorEmploymentVerificationCodeM,servicingAdvanceMethodCodeM,zeroBalanceCodeM,assetSubjectDemandStatusCodeM,repurchaseReplacementReasonCodeM,modificationTypeCodeM,accountStatus
0,CIK number-Sequential asset number,"=""0001694010 - 006770""",02-01-2017,02-28-2017,GM FINANCIAL,05/2012,30246.52,61,12/2017,0.18,1,1,0,06/2012,True,1,2,0,FORD,F350,2,2008,2,27700.0,98,Credit Bureau Score,594,2,2,True,0.0761,LA,False,0,False,1,9408.41,0.00,0.18,NaN,0.0225,0.0,0.0,0.0,44.406143,729.333857,0.0,0.00,773.74,9632.69,185.59,9408.41,38.69,0.0,-,02/2017,1,NaN,GM FINANCIAL,-,False,-,NaN,-,-,-,0.0,0.0,-,0,False,0.0,2017.03.23_AmeriCredit Automobile Receivables Trust 2017-1 Data Tape.csv,0001694010 - 006770,2017-02-01,2017-02-28,2012-05,2017-12,2012-06,,2017-02,,,Simple,Fixed,Monthly,No,Used,Truck,Other,"Stated, not verified","Stated, not verified",No advancing,Prepaid or Matured,Other,Other,Other,Prepaid or Matured
1,CIK number-Sequential asset number,"=""0001694010 - 006770""",12-01-2016,12-31-2016,GM FINANCIAL,05/2012,30246.52,61,12/2017,0.18,1,1,0,06/2012,True,1,2,0,FORD,F350,2,2008,2,27700.0,98,Credit Bureau Score,594,2,2,True,0.0761,LA,False,12,False,1,10640.12,773.74,0.18000000,0.18,0.0225,0.0,0.0,0.0,65.636964,708.103036,0.0,10033.93,773.74,747.48,141.29,606.19,0.00,0.0,12-15-2016,-,-,0.0,GM FINANCIAL,-,False,-,NaN,-,-,-,0.0,0.0,-,0,False,0.0,2017.02.07_AmeriCredit Automobile Receivables Trust 2017-1 Data Tape.csv,0001694010 - 006770,2016-12-01,2016-12-31,2012-05,2017-12,2012-06,2016-12-15,,,,Simple,Fixed,Monthly,No,Used,Truck,Other,"Stated, not verified","Stated, not verified",No advancing,Unavailable,Other,Other,Other,Unavailable


### Export

In [33]:
e_folder = 'data/static/'
e_file = '{} static.csv'.format(term)
e_path = e_folder + e_file
e_path

'data/static/AmeriCredit Automobile Receivables Trust 2017-1 Data Tape static.csv'

In [34]:
master.to_csv(e_path, index = False)

In [35]:
print('continue...')

continue...


### End